# 06. Object-Oriented Patterns in Kotlin

디자인 패턴에 대해 소개하는 그런 장인데 ... 솔직히 이 장은 그렇게 도움이 되지는 않을 듯

최대한 간단하게 여러가지 디자인 패턴을 소개하려 해서 모르는 사람은 이게뭔가 싶고 아는 사람은 그냥 너무 당연한 예시 같을거고 뭐 그런 느낌이다

그래서 책의 내용 대신 디자인 패턴과 관련된 것을 한두 개 소개하는 게 더 더 도움이 될 거 같다.

## Template method
- https://asvid.github.io/kotlin-template-method

이 노트북은 위 블로그 글에 나오는 코드를 기본 바탕으로 하여 약간씩 적절히 변형한 것이다.

참고로, 피자 예제는 이 블로그 글 말고도 다른 OOP관련 책이나 자료 등에서도 데코레이터 등의 패턴을 설명할 때도 단골로 사용하는 테마의 예제임

template method를 이해하면 factory method도 생성자에 대한 template method에 해당하는 개념이고, 또 책에 나오는 strategy도 template method의 특별한 사례로 볼 수도 있으니 ...

***주의***: 이런 피자 예제 같은 것들이 설명하는 디자인 패턴을 적용하는 것이 매우 적합한 상황이라고 오해하면 안된다.
그냥 그 개념을 설명하기 위해 OOP를 실무에서 본격적으로 많이 해보지 않은 사람도 이해하기 좋은 주제로 예제를 작성하는 것.

### Pizza

In [ ]:
abstract class Pizza { // base class for all pizza types

    // 피자 만들기 template method
    fun make() { // steps are the same for every pizza
        makeDough()      // 피자도우 반죽
        applySauce()     // 소스 바르기
        addIngredients() // 토핑 재료 추가
        bake()           // 굽기
    }

    // default implementation for each step
    // concrete classes needs to override only the distinctive ones
    open fun makeDough() = println("making 30cm dough")
    open fun addIngredients() = println("adding cheese")
    open fun applySauce() = println("applying tomato sauce")
    open fun bake() = println("baking for 20 minutes")
}

In [ ]:
open class Pepperoni : Pizza() { // concrete type of pizza
    override fun addIngredients() { // overriding ingredients according to recipe
        println("adding salami")
        println("adding onion")
        println("adding cheese")
    }
}

// 우리나라는 보통 R L F 이렇게 가니까 (코스트코같이 그런 미국사이즈 피자 다루는 대형마트 말고는)
// dough size variation
class PepperoniRegular: Pepperoni() { override fun makeDough() = println("making 23cm dough") }
class PepperoniLarge:   Pepperoni() // default size dough
class PepperoniFamily:  Pepperoni() { override fun makeDough() = println("making 38cm dough") }

// but it's not controlling the process of making a pizza
// all other methods are left with default implementation

In [3]:
PepperoniRegular().make()

making 23cm dough
applying tomato sauce
adding salami
adding onion
adding cheese
baking for 20 minutes


In [4]:
PepperoniLarge().make()

making 30cm dough
applying tomato sauce
adding salami
adding onion
adding cheese
baking for 20 minutes


In [5]:
PepperoniFamily().make()

making 38cm dough
applying tomato sauce
adding salami
adding onion
adding cheese
baking for 20 minutes


### Pizza lambdas

In [ ]:
abstract class Pizza( // base class constructor is taking lambdas but provides default implementation
    val makeDough:      () -> Unit = { println("making 30cm dough") },
    val applySauce:     () -> Unit = { println("applying tomato sauce") },
    val addIngredients: () -> Unit = { println("adding cheese") },
    val bake:           () -> Unit = { println("baking for 20 minutes") }
) {
    fun make() { // unchanged template method
        makeDough() // calling lambda parameter from constructor that can be replaced
        applySauce()
        addIngredients()
        bake()
    }
}

In [ ]:
val pepperoniAddons: () -> Unit = {
    println("adding salami")
    println("adding onion")
    println("adding cheese")
}

class PepperoniRegular: Pizza(
    addIngredients = pepperoniAddons,
    makeDough = { println("making 23cm dough") }
)
class PepperoniLarge:   Pizza(  // default size dough
    addIngredients = pepperoniAddons
)
class PepperoniFamily:  Pizza(
    addIngredients = pepperoniAddons,
    makeDough = { println("making 38cm dough") }
)

In [8]:
PepperoniRegular().make()

making 23cm dough
applying tomato sauce
adding salami
adding onion
adding cheese
baking for 20 minutes


In [9]:
PepperoniLarge().make()

making 30cm dough
applying tomato sauce
adding salami
adding onion
adding cheese
baking for 20 minutes


In [10]:
PepperoniFamily().make()

making 38cm dough
applying tomato sauce
adding salami
adding onion
adding cheese
baking for 20 minutes


## 싱글톤 Singleton 

전체 프로그램에서 딱 하나밖에 없는 객체

Kotlin에서는 별도의 코드로 디자인 패턴으로 작성할 필요 없는 아니라 언어에서 직접 제공하는 기능으로 표현 가능

Kotlin에서는 `object`라는 키워드를 제공

In [ ]:
object Sing // Sing은 타입 이름임과 동시에 유일무이한 객체의 이름

In [12]:
"Hello" is String // "Hello"라는 문자열 객체는 String이라는 타입이다

true

In [13]:
Sing is Sing // Sing이라는 객체는 Sing이라는 타입이다

true

In [14]:
val anotherSing = Sing() // 생성자 호출 불가

Line_13.jupyter.kts (1:19 - 23) Expression 'Sing' of type 'Line_10_jupyter.Sing' cannot be invoked as a function. The function 'invoke()' is not found

# 널 오브젝트 패턴

크기가 0인 대상 또는 초기화되지 않은 상태를 null로 표현했을 때 생기는 불편함(코딩 실수 가능성, 귀찮은 예외처리)을 줄이기 위해

In [ ]:
// 덜 객체지향스럽게 null을 직접 사용하는 방법
data class Node(
        val item: Int,
        val left: Node?,
        val right: Node?) {
    fun size(): Int {
        val n1 = if (left ==null) 0 else left.size()
        val n2 = if (right==null) 0 else right.size()
        return 1 + n1 + n2
    }
}

typealias Tree = Node?

In [ ]:
val t0: Tree = null
val t1: Tree = Node(1, null, null)
val t2: Tree = Node(1, Node(2,null,null), null)

In [17]:
println(t0)
println(t1)
println(t2)

null
Node(item=1, left=null, right=null)
Node(item=1, left=Node(item=2, left=null, right=null), right=null)


In [18]:
println(t1.size())
println(t2.size())

1
2


In [19]:
println(t0.size()) // 이건 t0가 null이라서 안됨

Line_20.jupyter.kts (1:11 - 12) Only safe (?.) or non-null asserted (!!.) calls are allowed on a nullable receiver of type Line_14_jupyter.Tree /* = Line_14_jupyter.Node? */

In [20]:
// 일반적으로 t0, t1, t2가 null인지 아닌지 확실치 않으면 매번 확인해서 처리해 줘야
println( if (t0==null) 0 else t0.size() )
println( if (t1==null) 0 else t1.size() )
println( if (t2==null) 0 else t2.size() )

0
1
2


In [1]:
// 널오브젝트 패턴을 활용해 좀더 객체지향적으로
sealed interface Tree {
    fun size(): Int
    
    data class Node(
            val item: Int,
            val left: Tree,
            val right: Tree): Tree {
        override fun size() = 1 + left.size() + right.size()
    }
    object Null: Tree {
        override fun size() = 0
        override fun toString() = "Null"
    }
}

In [2]:
val t0: Tree = Tree.Null
val t1: Tree = Tree.Node(1, Tree.Null, Tree.Null)
val t2: Tree = Tree.Node(1, Tree.Node(2,Tree.Null,Tree.Null), Tree.Null)

In [3]:
println(t0.size())
println(t1.size())
println(t2.size())

0
1
2


# 위임 delegation

클래스를 설계하다 보면 기존에 정의된 클래스가
지금 설계하는 클래스에서 제공해야 되는 기능을 전부(혹은 일부)
구현하고 있는 경우가 있다.



In [10]:
// 쓰기 지우기 둘 다 되는 펜을 이미 만들어 놓았는데
open class PenWD() {
    open fun write() { println("쓰기") }
    open fun delete() { println("지우기") }
}

In [12]:
// 쓰기만 되는 펜
class PenW(): PenWD() {
    // 땜빵식 문제점 해결
    override fun delete() {
        throw Exception("하지마 하지마 지우지마")
    }
}

In [13]:
val penw1 = PenW()
penw1.write()
penw1.delete() // 호출 가능 의도하지 않은 기능까지 제공

쓰기


하지마 하지마 지우지마
java.lang.Exception: 하지마 하지마 지우지마
	at Line_64_jupyter$PenW.delete(Line_64.jupyter.kts:5)
	at Line_67_jupyter.<init>(Line_67.jupyter.kts:3)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:499)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:480)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:105)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:47)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
	at kotlin.script.

org.jetbrains.kotlinx.jupyter.exceptions.ReplEvalRuntimeException: org.jetbrains.kotlinx.jupyter.exceptions.ReplEvalRuntimeException: 하지마 하지마 지우지마

In [14]:
class PenWD() {
    fun write() { println("쓰기") }
    fun delete() { println("지우기") }
}

In [15]:
class PenW() { // 수동으로 위임하기
    private val mypen = PenWD()
    fun write() = mypen.write() // write의 기능을 mypen에 위임
}

## 코틀린에서 제공하는 위임delegation 기능
인터페이스와 by라는 키워드를 활용

In [17]:
// 인터페이스로 내가 필요한 기능을 명세
interface Writer {
    fun write()
}

In [19]:
class PenWD(): Writer {
    override fun write() { println("쓰기") }
    fun delete() { println("지우기") }
}

In [20]:
class PenW(): Writer by PenWD()